# 04 – Formatter (LLM Gate + JSON Schema)


Only call the LLM when needed. Prefer deterministic formatting for simple answers (like emails).


In [ ]:
//r "nuget: RestSharp, 112.1.0"
using semantic_kernel_app;
using System.Text.Json;
using System.Text.Json.Serialization;
using RestSharp;
using Microsoft.Extensions.Logging;

public class AnswerFormatter : IAnswerFormatter
{
    private readonly AppConfig _cfg;
    private readonly ILogger<AnswerFormatter> _log;
    public AnswerFormatter(AppConfig cfg, ILogger<AnswerFormatter> log) { _cfg = cfg; _log = log; }

    public async Task<(bool usedLlm, string text)> FormatAsync(QuerySpec spec, object rawData, CancellationToken ct = default)
    {
        // If it's just contact info, we can format deterministically
        if (spec.Intent == Intent.GetContactInfo && rawData is IEnumerable<Employee> list)
        {
            var lines = list.Select(e => $"{e.DisplayName}: {e.Email}");
            return (false, string.Join("\n", lines));
        }

        // Otherwise, call Ollama with a strict JSON schema request
        var client = new RestClient(_cfg.OllamaBaseUrl);
        var prompt = BuildPrompt(spec, rawData);

        var body = new {
            model = _cfg.OllamaModel,
            prompt,
            options = new { temperature = 0.2 },
            stream = false
        };
        var req = new RestRequest("/api/generate", Method.Post).AddJsonBody(body);
        try
        {
            var res = await client.ExecuteAsync(req, ct);
            if (!res.IsSuccessful) return (true, $"LLM error: {(int)res.StatusCode} {res.ErrorMessage}");
            var content = JsonDocument.Parse(res.Content!).RootElement;
            var txt = content.GetProperty("response").GetString() ?? "";
            var clean = ExtractJson(txt) ?? txt;
            return (true, clean.Trim());
        }
        catch (Exception ex)
        {
            return (true, "LLM exception: " + ex.Message);
        }
    }

    private static string BuildPrompt(QuerySpec spec, object data)
    {
        var json = JsonSerializer.Serialize(data);
        return $@"
You are a helpful formatter. Given the user's intent and the typed data, return **valid JSON** ONLY.

Intent: {spec.Intent}
Slots: {JsonSerializer.Serialize(spec.Slots)}

TypedData (JSON):
{json}

Output schema (MUST match exactly):
{{
  ""items"": [ {{ ""name"": ""string"", ""email"": ""string"" }} ],
  ""summary"": ""string""
}}
Return ONLY the JSON, no prose.";
    }

    private static string? ExtractJson(string s)
    {
        var start = s.IndexOf('{');
        var end = s.LastIndexOf('}');
        if (start >=0 && end >= start) return s[start:end-start+1];
        return null;
    }
}

// demo
var formatter = new AnswerFormatter(new AppConfig(), LoggerFactory.Create(b=>b.AddConsole()).CreateLogger<AnswerFormatter>());
var sample = new List<Employee> {
    new("Rick Sanchez","rick.sanchez@company.com","Engineering","Staff Engineer", new DateTime(2015,5,10)),
    new("Morty Smith","morty.smith@company.com","Engineering","Engineer I", new DateTime(2022,9,14)),
};
var (used, txt) = await formatter.FormatAsync(new QuerySpec(Intent.GetContactInfo, new Slots()), sample);
Console.WriteLine($"usedLlm={used}\n{txt}");
